# Preliminary Analysis 

Analysis of artificial aneurysms introduced in the RCA with ASI = 2, at 3 positions, with 5 sizes per position

In [1]:
import numpy as np
import vtk
from vtk.util import numpy_support as nps
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import os 

In [2]:
from AneurysmGeneration.utils.batch import * 


Gotta read in the simulation results 

In [3]:
# define suffix
proximal = ['p1']#, 'p2', 'p3', 'p4', 'p5']
medial = ['m1', 'm2', 'm3', 'm4', 'm5']
distal = ['d1', 'd2', 'd3', 'd4', 'd5']

# define path
results_path_base = 'Artificial/RCA/'
source_path_base = 'AneurysmGeneration/models/SKD0050/'

In [16]:
def extract_points(polydata):
    NoP = polydata.GetNumberOfPoints()
    points = np.zeros((NoP, 3))
    
    for i in range(NoP):
        points[i] = polydata.GetPoints().GetPoint(i)
        
    return points

In [ ]:

# loop over the files and pickle some results 
for suff in proximal: 
    cur_mesh_path = results_path_base + suff + '/all_results.vtp'
    meshreader = vtk.vtkXMLPolyDataReader()
    meshreader.SetFileName(cur_mesh_path)
    meshreader.Update()
    meshdata = meshreader.GetOutput()

    cur_model_path = source_path_base + 'SKD0050_baseline_model_modified_' + suff + '.vtp'
    modelreader = vtk.vtkXMLPolyDataReader()
    modelreader.SetFileName(cur_model_path)
    modelreader.Update()
    modeldata = modelreader.GetOutput()
    
    # we can use the same precomputed data structures for all source models because the 
    # point and cell data are not changed when the expansions are generated
    (face_to_points, _, _, NoP) = read_from_file("big_boy")
    
    
    # now we want to gather the set of all point locations in the source model, surface mesh 
    points_source = extract_points(modeldata)
    points_mesh = extract_points(meshdata)
    print meshreader
    print meshdata
    
    # now we gotta do the mapping of mesh points onto source points 
    # the argmin should give the closest source point's pointID for each point in the mesh
    distances = np.sum((points_mesh - points_source[:, np.newaxis])**2, axis=2)
    mapping = np.argmin(distances, axis=0)
    
    print 'points source', points_source
    print 'points mesh', points_mesh
    print mapping
    # perform selection over the mapping
    # We must select for the right face, using face_to_points 
    include_ids = np.isin(mapping, face_to_points[8])
    
    print include_ids
    
    # Next, we're going to map each mesh point to a centerline point 
    
    # We will use this axial position mapping and our aneurysm parameters to 
    # isolate our analysis into several different regions 
    
    #Initialize total arrays
    # total_tawss =np.zeros((1,NoP))
    
    # 

reading structures from pickle
------------------------------
vtkXMLPolyDataReader (0x7fd9aa0f12e0)
  Debug: Off
  Modified Time: 4973
  Reference Count: 2
  Registered Events: (none)
  Executive: 0x7fd9aa1d5c80
  ErrorCode: Undefined error: 0
  Information: 0x7fd9a7d15800
  AbortExecute: Off
  Progress: 1
  Progress Text: (None)
  FileName: Artificial/RCA/p1/all_results.vtp
  CellDataArraySelection: 0x7fd9a652ffe0
  PointDataArraySelection: 0x7fd9a7d319c0
  Stream: (none)
  TimeStep:0
  NumberOfTimeSteps:0
  TimeStepRange:(0,0)


vtkPolyData (0x7fd9a7ddd410)
  Debug: Off
  Modified Time: 8435
  Reference Count: 2
  Registered Events: (none)
  Information: 0x7fd9a7abdbc0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 8452
  Field Data:
    Debug: Off
    Modified Time: 7040
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 76077
  Number Of Cells: 152150
  Cell Data:
    